##介绍
TensorFlow Text提供了可与TensorFlow 2.0一起使用的与文本相关的类和操作的集合。该库可以执行基于文本的模型所需的常规预处理，并包括其他对核心TensorFlow不提供的对序列建模有用的功能。

在文本预处理中使用这些操作的好处是它们在TensorFlow图中完成。您无需担心训练中的标记化与推理时的标记化或管理预处理脚本不同。

##渴望执行
TensorFlow Text需要TensorFlow 2.0，并且与eager模式和graph模式完全兼容。

In [0]:
!pip install -q tensorflow-text

In [3]:
try:
  %tensorflow_version 2.x
except Exception:
  pass

import tensorflow as tf
import tensorflow_text as text

TensorFlow 2.x selected.


###统一码
大多数操作人员期望字符串使用UTF-8。如果您使用其他编码，则可以使用核心tensorflow转码op将代码转码为UTF-8。如果您的输入可能无效，您也可以使用相同的操作将字符串强制为结构上有效的UTF-8。

In [0]:
docs = tf.constant([u'Everything not saved will be lost.'.encode('UTF-16-BE'),u'Sad☹'.encode('UTF-16-BE')])
utf8_docs = tf.strings.unicode_transcode(docs,input_encoding='UTF-16-BE',output_encoding='UTF-8')

##代币化
令牌化是将字符串分解为令牌的过程。通常，这些标记是单词，数字和/或标点符号。

主要接口分别是Tokenizer和TokenizerWithOffsets，tokenize并且tokenize_with_offsets分别具有单一方法。现在有多个令牌生成器。这些工具TokenizerWithOffsets（扩展了Tokenizer）中的每个工具都包括用于将字节偏移量获取到原始字符串中的选项。这使调用者可以知道创建令牌的原始字符串中的字节。

所有令牌生成器都返回RaggedTensors，其令牌的最内层维度映射到原始单个字符串。结果，所得形状的等级增加了一个。如果您不熟悉它们，请查看参差不齐的张量指南。https://www.tensorflow.org/guide/ragged_tensors

##空白令牌生成器
这是一个基本的令牌生成器，用于在ICU定义的空白字符（例如，空格，制表符，换行）上分割UTF-8字符串。

In [5]:
tokenizer = text.WhitespaceTokenizer()
tokens = tokenizer.tokenize(['everything not saved will be lost.',u'Sad☹'.encode('UTF-8')])
print(tokens.to_list())

Instructions for updating:
`tf.batch_gather` is deprecated, please use `tf.gather` with `batch_dims=-1` instead.
[[b'everything', b'not', b'saved', b'will', b'be', b'lost.'], [b'Sad\xe2\x98\xb9']]


###UnicodeScriptTokenizer
该令牌生成器根据Unicode脚本边界拆分UTF-8字符串。使用的脚本代码与Unicode（ICU）UScriptCode值的国际组件相对应。请参阅：http://icu-project.org/apiref/icu4c/uscript_8h.html

实际上，这与相似，WhitespaceTokenizer最明显的区别是它将标点符号（USCRIPT_COMMON）与语言文本（例如USCRIPT_LATIN，USCRIPT_CYRILLIC等）分开，同时也将语言文本彼此分开。

In [6]:
tokenizer = text.UnicodeScriptTokenizer()
tokens = tokenizer.tokenize(['everything not saved will be lost.',u'Sad☹'.encode('UTF-8')])
print(tokens.to_list())

[[b'everything', b'not', b'saved', b'will', b'be', b'lost', b'.'], [b'Sad', b'\xe2\x98\xb9']]


###Unicode拆分
当对没有空格的语言进行标记化以分割单词时，通常仅按字符进行拆分，这可以使用内核中找到的unicode_split op 来完成。

In [8]:
tokens = tf.strings.unicode_split([u'仅今年前'.encode('UTF-8')],'UTF-8')
print(tokens.to_list())

[[b'\xe4\xbb\x85', b'\xe4\xbb\x8a', b'\xe5\xb9\xb4', b'\xe5\x89\x8d']]


###偏移量
在对字符串进行标记时，通常需要知道标记在原始字符串中的何处。因此，每个实现TokenizerWithOffsets的令牌生成器都有一个tokenize_with_offsets方法，该方法将返回字节偏移量和令牌。offset_starts列出每个令牌开始的原始字符串中的字节，而offset_limits列出每个令牌结束的字节。

In [9]:
tokenizer = text.UnicodeScriptTokenizer()
(tokens,offset_starts,offset_limits) = tokenizer.tokenize_with_offsets(['everything not saved will be lost.',u'Sad☹'.encode('UTF-8')])
print(tokens.to_list())
print(offset_starts.to_list())
print(offset_limits.to_list())

[[b'everything', b'not', b'saved', b'will', b'be', b'lost', b'.'], [b'Sad', b'\xe2\x98\xb9']]
[[0, 11, 15, 21, 26, 29, 33], [0, 3]]
[[10, 14, 20, 25, 28, 33, 34], [3, 6]]


###TF.Data示例
标记程序可以与tf.data API一起正常工作。下面提供了一个简单的示例。

In [10]:
docs = tf.data.Dataset.from_tensor_slices([['Never tell me the odds.'],["It's a trap!"]])
tokenizer = text.WhitespaceTokenizer()
tokenized_docs = docs.map(lambda x:tokenizer.tokenize(x))
iterator = iter(tokenized_docs)
print(next(iterator).to_list())
print(next(iterator).to_list())

[[b'Never', b'tell', b'me', b'the', b'odds.']]
[[b"It's", b'a', b'trap!']]


###其他文字操作
TF.Text打包了其他有用的预处理操作。我们将在下面复习一些。

###字形
一些自然语言理解模型中使用的一个共同特征是查看文本字符串是否具有特定属性。例如，断句模型可能包含检查单词大写或标点字符是否在字符串末尾的功能。

Wordshape定义了各种有用的基于正则表达式的帮助器函数，用于匹配输入文本中的各种相关模式。这里有一些例子。

In [12]:
tokenizer = text.WhitespaceTokenizer()
tokens = tokenizer.tokenize(['Everything not saved will be lost.',u'Sad☹'.encode('UTF-8')])

f1 = text.wordshape(tokens,text.WordShape.HAS_TITLE_CASE)
f2 = text.wordshape(tokens,text.WordShape.IS_UPPERCASE)
f3 = text.wordshape(tokens,text.WordShape.HAS_SOME_PUNCT_OR_SYMBOL)
f4 = text.wordshape(tokens,text.WordShape.IS_NUMERIC_VALUE)

print(f1.to_list())
print(f2.to_list())
print(f3.to_list())
print(f4.to_list())

[[True, False, False, False, False, False], [True]]
[[False, False, False, False, False, False], [False]]
[[False, False, False, False, False, True], [True]]
[[False, False, False, False, False, False], [False]]


###N克和滑动窗口
N-gram是给定滑动窗口大小n的连续单词。组合令牌时，支持三种还原机制。对于文本，您可能想使用Reduction.STRING_JOIN将字符串彼此附加的字符串。默认的分隔符是空格，但是可以使用string_separater参数更改。

其他两种归约方法最常用于数值，它们是Reduction.SUM和Reduction.MEAN。

In [13]:
tokenizer = text.WhitespaceTokenizer()
tokens = tokenizer.tokenize(['Everything not saved will be lost',u'Sad☹'.encode('UTF-8')])

bigrams = text.ngrams(tokens,2,reduction_type=text.Reduction.STRING_JOIN)

print(bigrams.to_list())

[[b'Everything not', b'not saved', b'saved will', b'will be', b'be lost'], []]
